In [28]:
import polars as pl

In [29]:
def readAsDF(filename):
    df = pl.read_csv(
        file=filename,
        encoding="Windows 1252",
        sep=";",
        columns=[0,1],
        has_header=False,
    )
    headers = df.columns # headers
    df = df.rename({headers[0]: 'spanish', headers[1]: 'frenchSound'}) # rename cols

    # add root, folder, filename cols
    df = df.with_column(pl.lit("Audio").alias('root'))
    df = df.with_column(pl.lit("").alias('folder'))
    df = df.with_column(pl.lit("").alias('filename'))

    # replace null and nan, drop duplicated rows
    df = df.fill_nan("")
    df = df.fill_null("")
    df = df.unique()

    """
    # add folder col
    #containsClase = vecSpanish.str.contains("Clase")
    #df = df.with_column(pl.lit(containsClase).alias('folder'))
    #df = df.with_column((pl.col("spanish").str.starts_with("Clase").alias("folder")))
    #df = df.with_column(pl.col('frenchSound').apply(lambda x: re.split(rx, x)).alias('filename'))
    #df = df.with_column(pl.col('frenchSound').str.replace(r'\$i', pl.col('frenchSound')).alias('result'))
    #df = df.with_column(pl.col('frenchSound').str.replace_all('\?', ".").alias('filename'))
    #df = df.with_column(pl.col('frenchSound').arr.eval(pl.all().str.replace_all("?", ".").alias('filename')))
    #df = df.select([pl.col("frenchSound").str.count_match("c").alias("filename")])
    """

    # remove chars
    df = df.with_columns(
        [
            pl.when(
                True
                ).then(
                    pl.col('frenchSound').str.replace_all(rf'[*"<>|?¿!¡/]', ".") + ".mp3"
                    ).otherwise(
                        "ERROR").alias("filename")
        ]
    )
    
    folderName = ""
    vecSpanish = df["spanish"]

    for i in range(len(vecSpanish)):

        if vecSpanish[i].startswith("Clase"):
            folderName = vecSpanish[i].replace(" ", "")
        else:
            df[i, "folder"] = folderName # remove those chars
    
    # remove clases
    df = df.filter(pl.col("spanish").str.starts_with("Clase") == False)

    return df

In [30]:
df = readAsDF("French.csv")
df

spanish,frenchSound,root,folder,filename
str,str,str,str,str
"""hola""","""salut""","""Audio""","""Clase1""","""salut.mp3"""
"""chocolate""","""chocolat""","""Audio""","""Clase1""","""chocolat.mp3"""
"""hospital""","""hôpital""","""Audio""","""Clase1""","""hôpital.mp3"""
"""urgente""","""urgent""","""Audio""","""Clase1""","""urgent.mp3"""
"""un animal""","""un animal""","""Audio""","""Clase1""","""un animal.mp3"""
"""es""","""c'est""","""Audio""","""Clase1""","""c'est.mp3"""
"""es un animal""","""c'est un anima...","""Audio""","""Clase1""","""c'est un anima..."
"""difícil""","""difficile""","""Audio""","""Clase1""","""difficile.mp3"""
"""es dificil""","""c'est difficil...","""Audio""","""Clase1""","""c'est difficil..."


In [31]:
print(df[len(df)-1, "frenchSound"])
print(df[len(df)-1, "filename"])

c'est quelque chose de difficile? non, mais c'est très necessaire de le faire maintenant
c'est quelque chose de difficile. non, mais c'est très necessaire de le faire maintenant.mp3


In [32]:
df = df.with_columns(
    [
        pl.when(
            pl.col("spanish").str.starts_with("Clase") == False
            ).then(
                pl.col('frenchSound').str.replace_all(rf'[*"<>|?¿!¡/]', ".") + ".mp3"
                ).otherwise(
                    "ERROR").alias("filename")
    ]
)

In [33]:
df.head()

spanish,frenchSound,root,folder,filename
str,str,str,str,str
"""hola""","""salut""","""Audio""","""Clase1""","""salut.mp3"""
"""chocolate""","""chocolat""","""Audio""","""Clase1""","""chocolat.mp3"""
"""hospital""","""hôpital""","""Audio""","""Clase1""","""hôpital.mp3"""
"""urgente""","""urgent""","""Audio""","""Clase1""","""urgent.mp3"""
"""un animal""","""un animal""","""Audio""","""Clase1""","""un animal.mp3"""


In [34]:
filename = "config.csv"
df = pl.read_csv(
        file=filename,
        encoding="Windows 1252",
        sep=";",
        has_header=True,
)
df["language"][0]
df

language,filenameCSV
str,str
"""french""","""French.csv"""
